<a href="https://colab.research.google.com/github/MarkoBrie/Image-Classification-with-CNN-and-data-augmentation/blob/main/Image_classification_with_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<a name='0'></a>
# Sommaire

- [ Sommaire](#0)
- [ 1 - Configurer le noyau et les dépendances nécessaires](#1)
- [ 2 - Préparation du jeu de donnée](#2)
  - [ 2.0 - Charger le jeu de donnée CSV avec catégorie](#2.0)
  - [ 2.1 - Charger les images](#2.1)
  - [ 2.2 - Charger le jeu de donnée CSV avec catégorie](#2.2)
  - [ 2.3 - Afficher quelques exemples d'images par catégories](#2.3)
- [ 3 - SIFT, Clustering et ARI](#3)
  - [ 3.1 - Détermination et affichage des descripteurs SIFT](#3.1)
  - [ 3.2 - Classification d'images](#3.2)
- [ 4 - Perform Fine-Tuning to Detoxify the Summaries](#4)
  - [ 4.1 - Création des features des images](#4.1)
  - [ 4.2 - Réduction de dimension et analyse](#4.2)
    - [ 4.2.1 - Réduction de dimension avec PCA](#4.2.1)
    - [ 4.2.2 - Réduction de dimension T-SNE et affichage des images selon vraies classes](#4.2.1)
      
  - [ 3.4 - Evaluate the Model Qualitatively](#3.4)

<a name='1'></a>
# 1 - Configurer le noyau et les dépendances nécessaires

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import os
from os import listdir
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, auc, roc_auc_score, roc_curve
from glob import glob

from sklearn import preprocessing

import tensorflow as tf
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import GlobalAveragePooling2D, GlobalAveragePooling1D, Flatten, Dense, Dropout
from tensorflow.keras.layers import Rescaling, RandomFlip, RandomRotation, RandomZoom
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.utils import to_categorical

# os.environ["TF_KERAS"]='1'
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  0


<a name='2'></a>
# 2 Préparation du jeu de donnée

<a name='2.0'></a>
## 2.0 Télécharger le jeu de donnée zipped et unzip

In [ ]:
import requests
import zipfile
import io
import os

# URL of the zip file
zip_url = 'https://s3-eu-west-1.amazonaws.com/static.oc-static.com/prod/courses/files/Parcours_data_scientist/Projet+-+Textimage+DAS+V2/Dataset+projet+pre%CC%81traitement+textes+images.zip'

# Destination folder to extract the files
destination_folder = './'

# Create the destination folder if it doesn't exist
os.makedirs(destination_folder, exist_ok=True)

# Download the zip file
response = requests.get(zip_url)

# Check if the download was successful
if response.status_code == 200:
    # Unzip the content
    with zipfile.ZipFile(io.BytesIO(response.content)) as zip_ref:
        zip_ref.extractall(destination_folder)
    print("Zip file successfully downloaded and extracted.")
else:
    print(f"Failed to download the zip file. Status code: {response.status_code}")

Zip file successfully downloaded and extracted.


<a name='2.1'></a>
## 2.1 Charger les images

In [ ]:
path = "./Flipkart/Images"
print(path)

data_path = os.listdir(path)
#print(data_path)

<a name='2.2'></a>
## 2.2 Charger le jeu de donnée CSV avec catégorie

In [ ]:
# Specify the path to your CSV file
#file_path = './Flipkart/flipkart_com-ecommerce_sample_1050_with_CAT.csv'
file_path = './Flipkart/flipkart_com-ecommerce_sample_1050.csv'

# Read the CSV file into a pandas DataFrame
df = pd.read_csv(file_path)

# Display the first few rows of the DataFrame
#print(df.head(2))

In [ ]:
def cat_seperator_fct(sentence):
    sentence_clean = sentence.replace('["','')
    first_cat = sentence_clean.split(" >> ")[0]
    return first_cat

df['product_category_tree_1'] = df['product_category_tree'].apply(cat_seperator_fct)
df['product_category_tree_1']

In [ ]:
# L'éxtraction des noms de catégorie
l_cat = list(set(df['product_category_tree_1']))
print("catégories : ", l_cat)

list_labels = l_cat
label_no_name = "no_name "
print('Number of categories : ', len(l_cat))

In [ ]:
df['image_path'] = './Flipkart/Images/' + df['image']
df['image_dir'] = './Flipkart/Images/'

In [ ]:
le = preprocessing.LabelEncoder()
le.fit(df["product_category_tree_1"])
df["label"] = le.transform(df["product_category_tree_1"])

In [ ]:
df

<a name='2.3'></a>
## 2.3 Afficher quelques exemples d'images par catégories

In [ ]:
from matplotlib.image import imread

for name in list_labels :
    print(name)
    img_test = df[df['product_category_tree_1']==name]['image'][:3]
    #img_test
    # print("-------")
    for i in range(3):
        plt.subplot(130 + 1 + i)
        #filename = list_fct(name)[i+10]
        print(i, ' ', img_test.iloc[i])
        filename = './Flipkart/Images/'+img_test.iloc[i]
        image = imread(filename)
        plt.imshow(image)
    plt.show()

<a name='3'></a>
# 3 SIFT, clustering et ARI

<a name='3.1'></a>
## 3.1 Détermination et affichage des descripteurs SIFT d'un image

* L'image contient 319 descripteurs
* Chaque descripteur est un vecteur de longueur 128

In [ ]:
# In order to make xfeatures2d available run the following code
#pip install opencv-contrib-python

In [ ]:
# Import the OpenCV library
import cv2

# Create an instance of the SIFT (Scale-Invariant Feature Transform) detector
sift = cv2.xfeatures2d.SIFT_create()


# Read an image from the specified path (using the second image in the list_photos)
file = df["image_path"][3]
image = cv2.imread(file, 0)  # Convert the image to grayscale
image2show = imread(file)
plt.imshow(image2show)

plt.show()

# Equalize the histogram of the image to enhance contrast
image = cv2.equalizeHist(image)

# Detect and compute SIFT keypoints and descriptors
kp, des = sift.detectAndCompute(image, None)
keyp = sift.detect(image, None)

# Draw the detected keypoints on the image
img = cv2.drawKeypoints(image, keyp, image)

# Display the image with keypoints
plt.imshow(img)
plt.show()

# Print information about the computed descriptors
print("Descripteurs : ", des.shape)
print()
print(des)

<a name='3.2'></a>
## 3.2 Classification d'images avec SIFT, bag-of-features et classification supervisée

<figure>
<img src="https://user.oc-static.com/upload/2018/03/13/15209802570047_methode_classif.png" alt="AnVIL Portal Image."/>
<figure-caption>Figure. Les trois étapes d'un algorithme de classification d'images.</figure-caption>
</figure>

Etape 1 : Extraction des features. On utilise seulement SIFT pour détecter et décrire les features.  
Etape 2 : Création des bag-of-features.  
Etape 3 : Classification Supervisée

<a name='3.3'></a>
## Pré-traitement des images via SIFT
### ETAPE 1: Créations des descripteurs de chaque image
* Pour chaque image passage en gris et equalisation
* création d'une liste de descripteurs par image ("sift_keypoints_by_img") qui sera utilisée pour réaliser les histogrammes par image
* création d'une liste de descripteurs pour l'ensemble des images ("sift_keypoints_all") qui sera utilisé pour créer les clusters de descripteurs

In [ ]:
num_photos = len(df["image_path"])
print("working on ", num_photos, " photos")

In [ ]:
# identification of key points and associated descriptors
import time, cv2
sift_keypoints = []
temps1=time.time()
sift = cv2.xfeatures2d.SIFT_create(500)

for image_num in range(num_photos) :
    if image_num%100 == 0 : print(image_num)
    image = cv2.imread(df["image_path"][image_num],0) # convert in gray
    # image = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
    res = cv2.equalizeHist(image)   # equalize image histogram
    kp, des = sift.detectAndCompute(res, None)
    sift_keypoints.append(des)

sift_keypoints_by_img = np.asarray(sift_keypoints)
sift_keypoints_all    = np.concatenate(sift_keypoints_by_img, axis=0)

print()
print("Nombre de descripteurs : ", sift_keypoints_all.shape)

duration1=time.time()-temps1
print("temps de traitement SIFT descriptor : ", "%15.2f" % duration1, "secondes")

## ETAPE 2: Création des clusters de descripteurs (bag-of-features)
La création du "dictionnaire" des clusters visuels, se réalise en appliquant un algorithme de clustering aux descripteurs de features construits à l'étape 1, comme le k-means. Les visual words correspondent alors aux centres des clusters trouvés.

* Utilisation de MiniBatchKMeans pour obtenir des temps de traitement raisonnables

In [ ]:
from sklearn import cluster, metrics

# Determination number of clusters
temps1=time.time()

k = int(round(np.sqrt(len(sift_keypoints_all)),0))
print("Nombre de clusters estimés : ", k)
print("Création de",k, "clusters de descripteurs ...")

# Clustering
kmeans = cluster.MiniBatchKMeans(n_clusters=k, init_size=3*k, random_state=0)
kmeans.fit(sift_keypoints_all)

duration1=time.time()-temps1
print("temps de traitement kmeans : ", "%15.2f" % duration1, "secondes")

## ETAPE 3: CLASSIFICATION SUPERVISEE


* Pour chaque image :
   - prédiction des numéros de cluster de chaque descripteur
   - création d'un histogramme = comptage pour chaque numéro de cluster du nombre de descripteurs de l'image

Features d'une image = Histogramme d'une image = Comptage pour une image du nombre de descripteurs par cluster

In [ ]:
# Creation of histograms (features)
temps1=time.time()

def build_histogram(kmeans, des, image_num):
    res = kmeans.predict(des)
    hist = np.zeros(len(kmeans.cluster_centers_))
    nb_des=len(des)
    if nb_des==0 : print("problème histogramme image  : ", image_num)
    for i in res:
        hist[i] += 1.0/nb_des
    return hist


# Creation of a matrix of histograms
hist_vectors=[]

for i, image_desc in enumerate(sift_keypoints_by_img) :
    if i%100 == 0 : print(i)
    hist = build_histogram(kmeans, image_desc, i) #calculates the histogram
    hist_vectors.append(hist) #histogram is the feature vector

im_features = np.asarray(hist_vectors)

duration1=time.time()-temps1
print("temps de création histogrammes : ", "%15.2f" % duration1, "secondes")

## Réductions de dimension
### Réduction de dimension PCA
* La réduction PCA permet de créer des features décorrélées entre elles, et de diminuer leur dimension, tout en gardant un niveau de variance expliquée élevé (99%)
* L'impact est une meilleure séparation des données via le T-SNE et une réduction du temps de traitement du T-SNE

In [ ]:
from sklearn import manifold, decomposition

print("Dimensions dataset avant réduction PCA : ", im_features.shape)
pca = decomposition.PCA(n_components=0.99)
feat_pca= pca.fit_transform(im_features)
print("Dimensions dataset après réduction PCA : ", feat_pca.shape)

### Réduction de dimension T-SNE
* Réduction de dimension en 2 composantes T-SNE pour affichage en 2D des images

In [ ]:
from sklearn import manifold, decomposition

tsne = manifold.TSNE(n_components=2, perplexity=30,
                     n_iter=2000, init='random', random_state=6)
X_tsne = tsne.fit_transform(feat_pca)

df_tsne = pd.DataFrame(X_tsne[:,0:2], columns=['tsne1', 'tsne2'])
df_tsne["class"] = df["product_category_tree_1"] #data["label_name"]
print(df_tsne.shape)

In [ ]:
plt.figure(figsize=(15,10))
sns.scatterplot(
    x="tsne1", y="tsne2", hue="class", data=df_tsne, legend="brief",
    palette=sns.color_palette('tab10', n_colors=7), s=50, alpha=0.6)

plt.title('TSNE selon les vraies classes', fontsize = 30, pad = 35, fontweight = 'bold')
plt.xlabel('tsne1', fontsize = 26, fontweight = 'bold')
plt.ylabel('tsne2', fontsize = 26, fontweight = 'bold')
plt.legend(prop={'size': 14})

plt.show()

## Analyse mesures : similarité entre catégories et clusters

In [ ]:
from sklearn import cluster, metrics

cls = cluster.KMeans(n_clusters=7, random_state=6)
cls.fit(X_tsne)

df_tsne["cluster"] = cls.labels_
print(df_tsne.shape)

###  Affichage des images selon clusters et calcul ARI de similarité catégories images / clusters
* Le score ARI de 0.03 est très faible

In [ ]:
plt.figure(figsize=(10,6))
sns.scatterplot(
    x="tsne1", y="tsne2",
    hue="cluster",
    palette=sns.color_palette('tab10', n_colors=7), s=50, alpha=0.6,
    data=df_tsne,
    legend="brief")

plt.title('TSNE selon les clusters', fontsize = 30, pad = 35, fontweight = 'bold')
plt.xlabel('tsne1', fontsize = 26, fontweight = 'bold')
plt.ylabel('tsne2', fontsize = 26, fontweight = 'bold')
plt.legend(prop={'size': 14})

plt.show()

labels = df["label"]
print("ARI : ", metrics.adjusted_rand_score(labels, cls.labels_))

### Analyse par classes


In [ ]:
df_tsne.groupby("cluster").count()["class"]

In [ ]:
conf_mat = metrics.confusion_matrix(labels, cls.labels_)
print(conf_mat)

In [ ]:
def conf_mat_transform(y_true,y_pred) :
    conf_mat = metrics.confusion_matrix(y_true,y_pred)

    # corresp = np.argmax(conf_mat, axis=0)
    corresp = [6, 5, 4, 3, 1, 2, 0]
    print ("Correspondance des clusters : ", corresp)
    # y_pred_transform = np.apply_along_axis(correspond_fct, 1, y_pred)
    labels = pd.Series(y_true, name="y_true").to_frame()
    labels['y_pred'] = y_pred
    labels['y_pred_transform'] = labels['y_pred'].apply(lambda x : corresp[x])

    return labels['y_pred_transform']

cls_labels_transform = conf_mat_transform(labels, cls.labels_)
conf_mat = metrics.confusion_matrix(labels, cls_labels_transform)
print(conf_mat)
print()
print(metrics.classification_report(labels, cls_labels_transform))

In [ ]:
df_cm = pd.DataFrame(conf_mat, index = [label for label in list_labels],
                  columns = [i for i in "0123456"])
plt.figure(figsize = (6,4))
sns.heatmap(df_cm, annot=True, cmap="Blues")

<a name='4'></a>
# 4 Etape 1 : étude de faisabilité d'une classification supervisée



<a name='4.0'></a>
## 4.0 Création du modéle préentrainé basé sur un VGG16

In [ ]:
# Import the VGG16 model, a pre-trained convolutional neural network for image classification.
# The model is imported from the Keras applications module.
base_model = VGG16()

# Create a new model that takes the same inputs as the VGG16 model
# but outputs the second-to-last layer's output (penultimate layer)
model = Model(inputs=base_model.inputs, outputs=base_model.layers[-2].output)

# Print a summary of the architecture and parameters of the newly created model
#print(model.summary())

<a name='4.1'></a>
## 4.1 Création des features des images

In [ ]:
# Create an empty list to store the extracted features from images
images_features = []

# Initialize a counter for tracking progress
i = 0

# Iterate through each image file path in the "image_path" column of the "data" DataFrame
for image_file in df["image_path"]:
    # Print progress every 100 images
    if i % 100 == 0:
        print(i)

    # Increment the counter
    i += 1

    # Load the image, resizing it to the target size of (224, 224)
    image = load_img(image_file, target_size=(224, 224))

    # Convert the image to a numpy array
    image = img_to_array(image)

    # Add an extra dimension to the image array to match the input shape expected by the model
    image = np.expand_dims(image, axis=0)

    # Preprocess the image data to align with the preprocessing used during training the model
    image = preprocess_input(image)

    # Use the pre-trained model to predict features from the preprocessed image
    # Append the predicted features to the list
    images_features.append(model.predict(image, verbose=0)[0])

# Convert the list of image features to a numpy array
images_features = np.asarray(images_features)

# Print the shape of the resulting array (number of images x number of features)
print(images_features.shape)

<a name='4.2'></a>
## 4.2 Réduction dimension et analyse

<a name='4.2.1'></a>
### 4.2.1 Réduction de dimension PCA

In [ ]:
from sklearn import manifold, decomposition

print(images_features.shape)
pca = decomposition.PCA(n_components=0.99)
feat_pca= pca.fit_transform(images_features)
print(feat_pca.shape)

<a name='4.2.2'></a>
### 4.2.2 Réduction de dimension T-SNE et affichage des images selon vraies classes

In [ ]:
import time

temps1 = time.time()

tsne = manifold.TSNE(n_components=2, perplexity=30, n_iter=2000, init='random', random_state=6)
X_tsne = tsne.fit_transform(feat_pca)

duration1=time.time()-temps1
print("temps de T-SNE : ", "%15.2f" % duration1, "secondes")

In [ ]:
df_tsne = pd.DataFrame(X_tsne, columns=['tsne1', 'tsne2'])
df_tsne["class"] = df['product_category_tree_1'] # data["label_name"]

plt.figure(figsize=(15,10))
sns.scatterplot(
    x="tsne1", y="tsne2",
    hue="class",
    palette=sns.color_palette('tab10', n_colors=7), s=50, alpha=0.6,
    data=df_tsne,
    legend="brief")

plt.title('TSNE selon les vraies classes', fontsize = 30, pad = 35, fontweight = 'bold')
plt.xlabel('tsne1', fontsize = 26, fontweight = 'bold')
plt.ylabel('tsne2', fontsize = 26, fontweight = 'bold')
plt.legend(prop={'size': 14})

plt.show()

todo kmeans + ari

<a name='5'></a>
# 5 Etape 2 : Classification Supervisée
4 approches sont présentées :
* 1 Une approche simple par préparation initiale de l'ensemble des images avant classification supervisée avec un CNN
* 2 Une approche par data generator, permettant facilement la data augmentation. Les images sont directement récupérées à la volée dans le repertoire des images
* 3 Une approche récente proposée par Tensorflow.org par DataSet, sans data augmentation
* 4 Une approche par  DataSet, avec data augmentation intégrée au modèle : layer en début de modèle


<a name='5.0'></a>
## 5.0 Création du modèle de classification

In [ ]:
# Define a function to create a custom model based on VGG16
def create_model_fct():
    # Retrieve a pre-trained VGG16 model without the top classification layer
    model0 = VGG16(include_top=False, weights="imagenet", input_shape=(224, 224, 3))

    # Set all layers of the pre-trained model to non-trainable
    for layer in model0.layers:
        layer.trainable = False

    # Get the output of the pre-trained model
    x = model0.output


    # Add additional layers to the model
    # Part of the network that does the classification
    x = GlobalAveragePooling2D()(x)  # Global average pooling layer
    x = Dense(256, activation='relu')(x)  # Fully connected layer with ReLU activation
    x = Dropout(0.5)(x)  # Dropout layer to reduce overfitting
    predictions = Dense(7, activation='softmax')(x)  # Output layer with softmax activation

    # Define the new model with the added layers
    model = Model(inputs=model0.input, outputs=predictions)

    # Compile the model with categorical crossentropy loss, rmsprop optimizer, and accuracy metric
    model.compile(loss="categorical_crossentropy", optimizer='rmsprop', metrics=["accuracy"])

    # Print a summary of the model architecture
    print(model.summary())

    # Return the created model
    return model

In [ ]:
# evaluation metrics

# Step 1: Initialize an empty DataFrame
columns = ['Method', 'Training_Accuracy', 'Validation_Accuracy']
CNN_methods_accuracy = pd.DataFrame(columns=columns)

<a name='5.1'></a>
## 5.1 Approche préparation initiale des images

In [ ]:
# Specify the fraction of data to allocate to the second part (e.g., 0.2 for 20%)
test_size = 0.2

# Use train_test_split to randomly split the DataFrame
df_train, df_test = train_test_split(df, test_size=test_size, random_state=42)

df_train.reset_index(inplace=True)
df_test.reset_index(inplace=True)
# Display the shapes of the two parts
print("Shape of Part 1:", df_train.shape)
print("Shape of Part 2:", df_test.shape)

In [ ]:
def image_prep_fct(data) :
    prepared_images = []
    for image_num in range(len(data['image_path'])) :
        img = (load_img(
            data['image_path'][image_num],
            target_size=(224, 224)))
        img = img_to_array(img)
        img = img.reshape((img.shape[0], img.shape[1], img.shape[2]))
        img = preprocess_input(img)
        prepared_images.append(img)
        prepared_images_np = np.array(prepared_images)
    return prepared_images_np

images_np = image_prep_fct(df_train)
print(images_np.shape)
images_np_test = image_prep_fct(df_test)
print(images_np_test.shape)

In [ ]:
df_train.head(2)

In [ ]:
X = images_np
y = to_categorical(df_train['label'])

X_test = images_np_test
y_test = to_categorical(df_test['label'])

y

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X, y, stratify=y, test_size=0.25, random_state=42)
X_train.shape

In [ ]:
# Création du modèle
with tf.device('/gpu:0'):
    model1 = create_model_fct()

# Création du callback
model1_save_path1 = "./Output/model1_best_weights.h5"
checkpoint = ModelCheckpoint(model1_save_path1, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=5)
callbacks_list = [checkpoint, es]

In [ ]:
print(X_val.shape)
print(y_val.shape)
print(X_train.shape)
print(y_train.shape)

In [ ]:
# Entraîner sur les données d'entraînement (X_train, y_train)
with tf.device('/gpu:0'):
    history1 = model1.fit(X_train, y_train, epochs=50, batch_size=64,
                       callbacks=callbacks_list, validation_data=(X_val, y_val), verbose=1)


In [ ]:
# Score du dernier epoch

loss, train_accuracy = model1.evaluate(X_train, y_train, verbose=True)
print("Training Accuracy: {:.4f}".format(accuracy))
print()
loss, val_accuracy = model1.evaluate(X_val, y_val, verbose=True)
print("Validation Accuracy:  {:.4f}".format(accuracy))

new_values = {'Method': '1 base', 'Training_Accuracy': train_accuracy, 'Validation_Accuracy': val_accuracy}
CNN_methods_accuracy = CNN_methods_accuracy.append(new_values, ignore_index=True)


In [ ]:
# Score de l'epoch optimal

model1.load_weights(model1_save_path1)

loss, accuracy = model1.evaluate(X_val, y_val, verbose=False)
print("Validation Accuracy :  {:.4f}".format(accuracy))

loss, accuracy = model1.evaluate(X_test, y_test, verbose=False)
print("Test Accuracy       :  {:.4f}".format(accuracy))


In [ ]:
#pip install plot_keras_history


In [ ]:
from plot_keras_history import show_history, plot_history
import matplotlib.pyplot as plt

show_history(history1)
plot_history(history1, path="standard.png")
plt.close()

In [ ]:
y_val_num = np.argmax(y_val, axis=1)
y_val_pred = np.argmax(model1.predict(X_val), axis=1)
y_val_num = np.argmax(y_val, axis=1)
print(y_val_num)
print()
print(y_val_pred)

In [ ]:
conf_mat = metrics.confusion_matrix(y_val_num, y_val_pred)
print(conf_mat)

In [ ]:
y_val_pred_transform = conf_mat_transform(y_val_num, y_val_pred)
conf_mat = metrics.confusion_matrix(y_val_num, y_val_pred_transform)
print(conf_mat)
print()
print(metrics.classification_report(y_val_num, y_val_pred_transform))

In [ ]:
df_cm = pd.DataFrame(conf_mat, index = [label for label in list_labels],
                  columns = [i for i in "0123456"])
plt.figure(figsize = (6,7))
sns.heatmap(df_cm, annot=True, cmap="Blues")

<a name='5.2'></a>
## 5.2 Approche ImageDatagenerator avec data augmentation

CF https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/image/ImageDataGenerator#flow (noté désormais comme "deprecated", incite à utiiser l'approche suivante)

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
# Pour mélanger les images, classées initalement par classe
data = df.sample(frac=1, random_state=42).reset_index(drop=True)

In [ ]:
batch_size = 32

def data_flow_fct(data, datagen, data_type=None) :
    data_flow = datagen.flow_from_dataframe(data, directory='',
                                x_col='image_path', y_col='product_category_tree_1',
                                weight_col=None, target_size=(256, 256),
                                classes=None, class_mode='categorical',
                                batch_size=batch_size, shuffle=True, seed=42,
                                subset=data_type
                                )
    return data_flow

In [ ]:
datagen_train = ImageDataGenerator(
#    featurewise_center=True,
#    featurewise_std_normalization=True,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    validation_split=0.25,
    preprocessing_function=preprocess_input)

train_flow = data_flow_fct(data, datagen_train, data_type='training')
val_flow = data_flow_fct(data, datagen_train, data_type='validation')

datagen_test = ImageDataGenerator(
    validation_split=0,
    preprocessing_function=preprocess_input)

test_flow = data_flow_fct(df_test, datagen_test, data_type=None)

# compute quantities required for featurewise normalization
# (std, mean, and principal components if ZCA whitening is applied)
# datagen.fit(X_train)
# fits the model on batches with real-time data augmentation:

In [ ]:
# Création du modèle
with tf.device('/gpu:0'):
    model2 = create_model_fct()

# Création du callback
model2_save_path = "./model2_best_weights.h5"
checkpoint = ModelCheckpoint(model2_save_path, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=5)
callbacks_list = [checkpoint, es]


In [ ]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))


In [ ]:
with tf.device('/gpu:0'):
    history2 = model2.fit(train_flow,
                    validation_data=val_flow,
                    batch_size=batch_size, epochs=50, callbacks=callbacks_list, verbose=1)


In [ ]:
# Score du dernier epoch

loss, accuracy = model2.evaluate(train_flow, verbose=True)
print("Training Accuracy   : {:.4f}".format(accuracy))
print()
loss, accuracy = model2.evaluate(val_flow, verbose=True)
print("Validation Accuracy :  {:.4f}".format(accuracy))

In [ ]:
# Score de l'epoch optimal

model2.load_weights(model2_save_path)

loss, accuracy = model2.evaluate(val_flow, verbose=False)
print("Validation Accuracy :  {:.4f}".format(accuracy))

loss, accuracy = model2.evaluate(test_flow, verbose=False)
print("Test Accuracy       :  {:.4f}".format(accuracy))


In [ ]:
from plot_keras_history import show_history, plot_history
import matplotlib.pyplot as plt

show_history(history2)
plot_history(history2, path="standard.png")
plt.close()

<a name='5.3'></a>
## 5.3 Approche nouvelle par Dataset sans data augmentation

CF https://www.tensorflow.org/tutorials/load_data/images

In [ ]:

# import module
import shutil

In [ ]:
batch_size = 32

def dataset_fct(path, validation_split=0, data_type=None) :
    print(path)
    dataset = tf.keras.utils.image_dataset_from_directory(
                    path,
                    labels='inferred', label_mode='categorical',
                    #class_names=None,
                    batch_size=32, image_size=(224, 224), shuffle=True, seed=42,
                    validation_split=validation_split, subset=data_type
                    )
    return dataset

In [ ]:
try:
    os.mkdir('./Flipkart/TRAIN')
except OSError as error:
    print(error)

for i in class_folder_name:
    #print(i)
    try:
        os.mkdir('./Flipkart/TRAIN/'+str(i))
    except OSError as error:
        print(error)

for i in range(df_train.shape[0]):
    #print("i ", i)
    # copy the contents of the
    shutil.copyfile(df_train['image_path'][i], './Flipkart/TRAIN/'+df_train['product_category_tree_1'][i]+'/'+df_train['image'][i])


In [ ]:
class_folder_name = set(df['product_category_tree_1'])

try:
    os.mkdir('./Flipkart/TEST')
except OSError as error:
    print(error)

for i in class_folder_name:
    print(i)
    try:
        os.mkdir('./Flipkart/TEST/'+str(i))
    except OSError as error:
        print(error)

for i in range(df_test.shape[0]):
    #print("i ", i)
    # copy the contents of the
    shutil.copyfile(df_test['image_path'][i], './Flipkart/TEST/'+df_test['product_category_tree_1'][i]+'/'+df_test['image'][i])


In [ ]:
path_train = './Flipkart/TRAIN/'
path_test = './Flipkart/TEST/'

In [ ]:
dataset_train = dataset_fct(path_train, validation_split=0.25, data_type='training')
dataset_val = dataset_fct(path_train, validation_split=0.25, data_type='validation')
dataset_test = dataset_fct(path_test, validation_split=0, data_type=None)

In [ ]:

class_names = dataset_test.class_names
print(class_names)

In [ ]:
import matplotlib.pyplot as plt

print(class_names)
plt.figure(figsize=(10, 10))
for images, labels in dataset_test.take(1):
    for i in range(9):
        ax = plt.subplot(3, 3, i + 1)
        plt.imshow(images[i].numpy().astype("uint8"))
        # Convert the labels tensor to a NumPy array and then use it as an index
        label_index = tf.argmax(labels[i]).numpy()
        print(labels[i], 'label index ',label_index)
        plt.title(class_names[label_index])
        plt.axis("off")

In [ ]:
# Création du modèle
with tf.device('/gpu:0'):
    model3 = create_model_fct()

# Création du callback
model3_save_path = "./model3_best_weights.h5"
checkpoint = ModelCheckpoint(model3_save_path, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=5)
callbacks_list = [checkpoint, es]


In [ ]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))


In [ ]:
with tf.device('/gpu:0'):
    history3 = model3.fit(dataset_train,
                    validation_data=dataset_val,
                    batch_size=batch_size, epochs=50, callbacks=callbacks_list, verbose=1)


In [ ]:
# Score du dernier epoch

loss, accuracy = model3.evaluate(dataset_train, verbose=True)
print("Training Accuracy   : {:.4f}".format(accuracy))
print()
loss, accuracy = model3.evaluate(dataset_val, verbose=True)
print("Validation Accuracy :  {:.4f}".format(accuracy))

In [ ]:
# Score de l'epoch optimal

model3.load_weights(model3_save_path)

loss, accuracy = model3.evaluate(dataset_val, verbose=False)
print("Validation Accuracy :  {:.4f}".format(accuracy))

loss, accuracy = model3.evaluate(dataset_test, verbose=False)
print("Test Accuracy       :  {:.4f}".format(accuracy))


In [ ]:
from plot_keras_history import show_history, plot_history
import matplotlib.pyplot as plt

show_history(history3)
plot_history(history3, path="standard.png")
plt.close()


## 4.4 Approche nouvelle par Dataset avec data augmentation intégrée au modèle

CF https://www.tensorflow.org/tutorials/images/data_augmentation

In [ ]:
batch_size = 32

def dataset_fct(path, validation_split=0, data_type=None) :
    dataset = tf.keras.utils.image_dataset_from_directory(
                    path, labels='inferred', label_mode='categorical',
                    class_names=None,
                    batch_size=batch_size, image_size=(224, 224), shuffle=True, seed=42,
                    validation_split=validation_split, subset=data_type
                    )
    return dataset



In [ ]:
#dataset_train = dataset_fct(path_train, validation_split=0.25, data_type='training')
#dataset_val = dataset_fct(path_train, validation_split=0.25, data_type='validation')
#dataset_test = dataset_fct(path_test, validation_split=0, data_type=None)

In [ ]:
def resize_and_rescale(image, label):
    image = tf.cast(image, tf.float32)
    image = tf.image.resize(image, [IMG_SIZE, IMG_SIZE])
    image = (image / 255.0)
    return image, label

In [ ]:
def create_model_fct2() :
    # Data augmentation
    data_augmentation = Sequential([
        RandomFlip("horizontal", input_shape=(224, 224, 3)),
        RandomRotation(0.1),
        RandomZoom(0.1),
        # Rescaling(1./127.5, offset=-1.0)
      ])

    # Récupération modèle pré-entraîné
    model_base = VGG16(include_top=False, weights="imagenet", input_shape=(224, 224, 3))
    for layer in model_base.layers:
        layer.trainable = False

    # Définition du nouveau modèle
    model = Sequential([
                data_augmentation,
                Rescaling(1./127.5, offset=-1),
                model_base,
                GlobalAveragePooling2D(),
                Dense(256, activation='relu'),
                Dropout(0.5),
                Dense(7, activation='softmax')
                ])

    # compilation du modèle
    model.compile(loss="categorical_crossentropy", optimizer='adam', metrics=["accuracy"])

    print(model.summary())

    return model


# Création du modèle
with tf.device('/gpu:0'):
    model4 = create_model_fct2()

# Création du callback
model4_save_path = "./model4_best_weights.h5"
checkpoint = ModelCheckpoint(model4_save_path, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=5)
callbacks_list = [checkpoint, es]


In [ ]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

In [ ]:
with tf.device('/gpu:0'):
    history4 = model4.fit(dataset_train,
                    validation_data=dataset_val,
                    batch_size=batch_size, epochs=50, callbacks=callbacks_list, verbose=1)

In [ ]:
# Score du dernier epoch

loss, accuracy = model4.evaluate(dataset_train, verbose=True)
print("Training Accuracy   : {:.4f}".format(accuracy))
print()
loss, accuracy = model4.evaluate(dataset_val, verbose=True)
print("Validation Accuracy :  {:.4f}".format(accuracy))

In [ ]:
# Score de l'epoch optimal

model4.load_weights(model4_save_path)

loss, accuracy = model4.evaluate(dataset_val, verbose=False)
print("Validation Accuracy :  {:.4f}".format(accuracy))

loss, accuracy = model4.evaluate(dataset_test, verbose=False)
print("Test Accuracy       :  {:.4f}".format(accuracy))


In [ ]:
# Score du dernier epoch

loss, accuracy = model4.evaluate(dataset_train, verbose=True)
print("Training Accuracy   : {:.4f}".format(accuracy))
print()
loss, accuracy = model4.evaluate(dataset_val, verbose=True)
print("Validation Accuracy :  {:.4f}".format(accuracy))

In [ ]:
# Score de l'epoch optimal

model4.load_weights(model4_save_path)

loss, accuracy = model4.evaluate(dataset_val, verbose=False)
print("Validation Accuracy :  {:.4f}".format(accuracy))

loss, accuracy = model4.evaluate(dataset_test, verbose=False)
print("Test Accuracy       :  {:.4f}".format(accuracy))
